In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
cd ~/demres

/Users/zurfarosa/demres


In [26]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.common import codelists
from demres.common.process_pt_features import *
from demres.common.process_entries import *
from demres.demins.constants import Study_Design as sd
from demres.demins.statistical_functions import *
from common.helper_functions import *
from pprint import pprint
from IPython.display import display

In [27]:
pd.set_option('display.max_columns', None)

In [28]:
pd.set_option('display.max_rows', None)

## Process raw CSV files

In [6]:
# create_pegmed()

In [7]:
# create_pegprod()

In [8]:
# create_prescriptions()

In [9]:
# create_consultations()

In [10]:
# create_clinicals()

In [11]:
# create_tests()

In [12]:
# create_referrals()

In [13]:
# create_immunisations()

In [14]:
# create_medcoded_entries()

## Create processed CVS files to share with DEMINS teams
*convert the newly created HDF files to CVS (for use in other projects)*

In [15]:
# prescriptions = pd.read_hdf('data/pt_data/processed_data/hdf/prescriptions.hdf')
# prescriptions.to_csv('data/pt_data/processed_data/to_share_with_DEMINS/prescriptions.csv',index=False)

In [16]:
# consultations = pd.read_hdf('data/pt_data/processed_data/hdf/consultations.hdf')
# consultations.to_csv('data/pt_data/processed_data/to_share_with_DEMINS/consultations.csv',index=False)

In [17]:
# immunisations = pd.read_hdf('data/pt_data/processed_data/hdf/immunisations.hdf')
# immunisations.to_csv('data/pt_data/processed_data/to_share_with_DEMINS/immunisations.csv',index=False)

In [18]:
# clinicals = pd.read_hdf('data/pt_data/processed_data/hdf/clinicals.hdf')
# clinicals.to_csv('data/pt_data/processed_data/to_share_with_DEMINS/clinicals.csv',index=False)

In [19]:
# tests = pd.read_hdf('data/pt_data/processed_data/hdf/tests.hdf')
# tests.to_csv('data/pt_data/processed_data/to_share_with_DEMINS/tests.csv',index=False)

In [20]:
# referrals = pd.read_hdf('data/pt_data/processed_data/hdf/referrals.hdf')
# referrals.to_csv('data/pt_data/processed_data/to_share_with_DEMINS/referrals.csv',index=False)

In [21]:
# medcoded_entries = pd.read_hdf('data/pt_data/processed_data/hdf/medcoded_entries.hdf')
# medcoded_entries.to_csv('data/pt_data/processed_data/to_share_with_DEMINS/medcoded_entries.csv',index=False)

## Create basic pt_features dataframe
*pt_features will contain all the variables (e.g. age, female gender, insomnia) used in the logistic regression*

In [23]:
all_encounters = get_all_encounters()

KeyboardInterrupt: 

In [ ]:
all_entries = get_all_entries(all_encounters)

In [ ]:
pt_features = create_pt_features()

In [ ]:
pt_features = get_index_date_and_caseness_and_add_final_dementia_subtype(all_entries,pt_features)

In [ ]:
pt_features.head(5)

In [ ]:
pt_features = avoid_specific_dementia_subtypes(pt_features) 

In [ ]:
pt_features = add_data_start_and_end_dates(all_encounters,pt_features)

In [ ]:
# pt_features.to_csv('data/pt_data/processed_data/pt_features_demins.csv',index=False)

In [ ]:
for window in sd.exposure_windows:
    print(window['name'],' being matched')    
    pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
    pt_features = match_cases_and_controls(pt_features,window)
    display(pt_features.head(1))
    pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+ window['name'] +'.csv',index=False)

## Add derived variables to pt_features 
*e.g. insomnia count, history of stroke, consultation count*

In [ ]:
medcoded_entries = pd.read_hdf('data/pt_data/processed_data/hdf/medcoded_entries.hdf')

In [29]:
prescriptions = pd.read_hdf('data/pt_data/processed_data/hdf/prescriptions.hdf')

In [30]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+sd.exposure_windows[1]['name']+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)

In [33]:
create_pdd_for_each_drug(prescriptions,pt_features,sd.exposure_windows[1])

['CLOMETHIAZOLE', 'DIAZEPAM', 'FLURAZEPAM', 'FLURAZEPAM HYDROCHLORIDE', 'LOPRAZOLAM', 'LOPRAZOLAM MESILATELORAZEPAM', 'LORMETAZEPAM', 'MELATONIN', 'NITRAZEPAM', 'OXAZEPAM', 'TEMAZEPAM', 'ZALEPLON', 'ZOLPIDEM TARTRATE', 'ZOLPIDEM ', 'ZOPICLONE']
CLOMETHIAZOLE 	pdd: 314.3554376657825
DIAZEPAM 	pdd: 4.661071466315938
FLURAZEPAM 	No prescriptions found
FLURAZEPAM HYDROCHLORIDE 	pdd: 29.39175572519084
LOPRAZOLAM 	No prescriptions found
LOPRAZOLAM MESILATELORAZEPAM 	No prescriptions found
LORMETAZEPAM 	pdd: 0.9058665784132888
MELATONIN 	pdd: 3.0
NITRAZEPAM 	pdd: 5.991664723363171
OXAZEPAM 	pdd: 20.785283646707338
TEMAZEPAM 	pdd: 14.246090863790956
ZALEPLON 	pdd: 7.073170731707317
ZOLPIDEM TARTRATE 	pdd: 7.135525904770405
ZOLPIDEM  	No prescriptions found
ZOPICLONE 	pdd: 6.4285431565793205


In [35]:
for window in sd.exposure_windows:
    print(window['name'],'...')
    pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+ window['name'] +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
#     pt_features = get_multiple_condition_statuses(pt_features,medcoded_entries,prescriptions,window,codelists.all_codelists)
    pt_features = create_PDD_columns_for_each_pt(pt_features,window,prescriptions)
#     pt_features = create_quantiles_and_booleans(pt_features)
    display(pt_features.head(1))    
    pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+ window['name'] +'.csv',index=False)

12_to_7 ...


TypeError: 'module' object is not iterable